In [1]:
%matplotlib
import yaml
import os
import torch
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()
torch.cuda.set_device(0)
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

Using matplotlib backend: <object object at 0x14fe48034ed0>


In [2]:
import sys
sys.argv = ["train.py"]
import logging
import torch
import argparse
import numpy as np
import yaml
import pickle
import random
from PIL import Image
from B_COS_eval import BCOSEvaluator
from LIME_eval import LIMEEvaluator  
# from GRADCAM_eval import GradCamEvaluator  # Uncomment if implemented.
from training.detectors.xception_detector import XceptionDetector
from training.detectors import DETECTOR
from dataset.abstract_dataset import DeepfakeAbstractBaseDataset

2025-03-27 11:00:24.937736: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-27 11:00:24.976710: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-27 11:00:24.976742: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-27 11:00:24.976766: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-27 11:00:24.985178: I tensorflow/core/platform/cpu_feature_g

In [3]:
torch.manual_seed(34)
random.seed(34)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False  # Disable for strict reproducibility


In [4]:
# Define default model configuration.
MODEL_PATH = os.path.join("/pfs/data5/home/ma/ma_ma/ma_lineuman/Interpretable-Deep-Fake-Detection", "training/config/detector/resnet34_bcos_v2_minimal.yaml")
ADDITIONAL_ARGS = {
    "test_batchSize": 12
}

In [5]:
# Setup logging.
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

In [6]:
def load_model(config):
    """Load and return the model using the DETECTOR registry."""
    logger.info("Registered models: %s", list(DETECTOR.data.keys()))
    model_class = DETECTOR[config['model_name']]
    model = model_class(config)
    return model

In [7]:
def load_config(path, additional_args={}):
    """Load and merge configuration files with any additional overrides."""
    with open(path, 'r') as f:
        config = yaml.safe_load(f)
    try:
        with open('./training/config/test_config.yaml', 'r') as f:
            config2 = yaml.safe_load(f)
    except FileNotFoundError:
        with open(os.path.expanduser('~/Interpretable-Deep-Fake-Detection/training/config/test_config.yaml'), 'r') as f:
            config2 = yaml.safe_load(f)
    if 'label_dict' in config:
        config2['label_dict'] = config['label_dict']
    config.update(config2)
    if config.get('dry_run', False):
        config['nEpochs'] = 0
        config['save_feat'] = False
    for key, value in additional_args.items():
        config[key] = value
    return config

In [8]:
def get_images_from_dataloader(data_loader):
    """Extract and return all images from a DataLoader."""
    all_images = []
    for batch in data_loader:
        images = batch['image']  # shape: [batch_size, C, H, W]
        for img in images:
            all_images.append(img)
    return all_images

In [9]:
def preprocess_image(img):
    # img is expected to be a tensor of shape [B, C, H, W] in a batch.
    if img.shape[1] == 3:
        img = torch.cat([img, 1.0 - img], dim=1)
    return img

In [10]:
class Analyser:
    def analysis(self):
        raise NotImplementedError("Need to implement analysis function.")

    def run(self):
        # Run analysis and save both raw and overall results.
        overall, raw = self.analysis()
        self.save_results(raw, overall)

    def save_results(self, raw, overall):
        """Save results to a designated folder."""
        save_folder = os.path.join("results", str(self.grid_split), f"{self.model_name}_{self.weights_name}")
        os.makedirs(save_folder, exist_ok=True)  # Create directory if needed.
        with open(os.path.join(save_folder, "results.pkl"), "wb") as f:
            pickle.dump(raw, f)  # Save raw results.
        with open(os.path.join(save_folder, "overall.pkl"), "wb") as f:
            pickle.dump(overall, f)  # Save overall metrics.
        logger.info("Results saved to folder: %s", save_folder)
        
    def load_results(self, load_overall=True):
        """Load results from disk and print summary info."""
        load_folder = os.path.join("results", str(self.grid_split), f"{self.model_name}_{self.weights_name}")
        file_path = os.path.join(load_folder, "overall.pkl" if load_overall else "results.pkl")
        with open(file_path, "rb") as f:
            loaded = pickle.load(f)
        logger.info("Results loaded from %s", file_path)
        if load_overall:
            localisation_metric = loaded.get("localisation_metric", None)
            percentiles = loaded.get("percentiles", None)
            if localisation_metric is not None and percentiles is not None:
                logger.info("Overall results: %d evaluations, percentiles: %s", len(localisation_metric), percentiles)
            else:
                logger.warning("Overall results missing expected keys.")
        else:
            sorted_results = sorted(loaded, key=lambda res: res.get("accuracy", 0), reverse=True)
            logger.info("Top raw results:")
            for idx, res in enumerate(sorted_results[:10]):
                logger.info("[%d] %s - Accuracy: %s", idx + 1, res.get("path", "N/A"), res.get("accuracy", "N/A"))
        return loaded

In [11]:
class RankedGPGCreator(Analyser):
    def __init__(self, base_output_dir, grid_size=(3, 3), xai_method=None, max_grids=3, plotting_only=False,
                 model=None, model_name="default", weights_name="default",
                 test_data_loaders=None, dataset=None, device=None, config=None, grid_split=3):
        """
        Initialize grid creator with specified parameters.
        base_output_dir: Base directory for grids.
        grid_size: Dimensions of the grid (e.g., (3,3)).
        xai_method: "bcos", "lime", or "gradcam".
        max_grids: Maximum number of grids to create.
        plotting_only: If True, load existing results.
        """
        self.grid_size = grid_size
        self.xai_method = xai_method
        self.max_grids = max_grids
        self.model = model
        self.test_data_loaders = test_data_loaders
        self.dataset = dataset
        self.model_name = model_name
        self.weights_name = weights_name
        self.output_folder = os.path.join(base_output_dir, f"{model_name}_{weights_name}")
        self.device = device
        self.grid_split = grid_split

        if plotting_only:
            self.load_results()
            return
        
        # Create output directory for grids.
        self.output_dir = os.path.join(self.output_folder, str(self.grid_split))
        os.makedirs(self.output_dir, exist_ok=True)

        # Load or compute sorted image rankings.
        self.ranking_file = os.path.join(self.output_folder, "sorted_confs.pkl")
        if os.path.exists(self.ranking_file):
            self.sorted_confs = self.load_ranking(self.ranking_file)
            logger.info("Loaded sorted confidences from %s", self.ranking_file)
        else:
            self.sorted_confs = self.compute_sorted_confs()
            self.save_ranking(self.sorted_confs, self.ranking_file)
            logger.info("Saved sorted confidences to %s", self.ranking_file)

    def compute_sorted_confs(self):
        """Compute ranking by storing (index, confidence) for each correctly classified image."""
        ranking = {0: [], 1: []}
        key = list(self.test_data_loaders.keys())[0]
        for i, data_dict in enumerate(self.test_data_loaders[key]):
            print(key)
            img_batch, label_batch, mask, landmark, index = (data_dict[k] for k in ['image', 'label', 'mask', 'landmark', 'index'])
            print(f"Batch of images shape: {img_batch.shape}")
          
            if 'label_spe' in data_dict:
                data_dict.pop('label_spe')  # remove the specific label
            data_dict['label'] = torch.where(data_dict['label']!=0, 1, 0)  # fix the label to 0 and 1 only
            for key, value in data_dict.items():
                # Skip if the value is a tuple (like the indices) or doesn't have a .to() method.
                if value is not None and hasattr(value, 'to'):
                    data_dict[key] = value.to(self.device)
    
            num_batches = img_batch.shape[0]
            # Iterate through each image in the batch
            for i in range(num_batches):
                image = img_batch[i].unsqueeze(0).to(self.device)
                label = label_batch[i]
                img_idx= index[i]
                
                if self.xai_method == "bcos":
                    image = preprocess_image(image)
                output = self.model({'image': image, 'label': label})
                logit = output['cls']
                
                predicted_label = logit[i].argmax().item()
                if label == 1:
                    if label == predicted_label:
                        # Append tuple (index, confidence) for correct classification.
                        confidence = logit[i, predicted_label].item()
                        ranking[label].append((img_idx, confidence))
                if label == 0:
                    if label == predicted_label:
                        confidence = logit[predicted_label].item()
                        ranking[label].append((img_idx, confidence))
        for cls in ranking:
            ranking[cls] = sorted(ranking[cls], key=lambda x: x[1], reverse=True)
            logger.debug("Class %d: %d images after sorting.", cls, len(ranking[cls]))
        return ranking

    '''
    def compute_sorted_confs(self):
        """Compute ranking by storing (index, confidence) for each correctly classified image."""
        ranking = {0: [], 1: []}
        img_idx = 0
        with torch.no_grad():
            # Iterate over data loader batches.
            for batch in self.loader:
                images = batch['image'].to(self.device)
                if self.xai_method == "bcos":
                    images = preprocess_image(images)
                labels = batch['label'].to(self.device)
                output = self.model({'image': images, 'label': labels})
                logits = output['cls']
                # Process each image in the batch.
                for i in range(len(images)):
                    true_label = int(labels[i].item())
                    predicted_label = logits[i].argmax().item()
                    if true_label == predicted_label:
                        # Append tuple (index, confidence) for correct classification.
                        confidence = logits[i, predicted_label].item()
                        ranking[true_label].append((img_idx, confidence))
                    img_idx += 1
        
        # Sort entries for each class by confidence (descending).
        for cls in ranking:
            ranking[cls] = sorted(ranking[cls], key=lambda x: x[1], reverse=True)
            logger.debug("Class %d: %d images after sorting.", cls, len(ranking[cls]))
        return ranking 
    '''
    
    def get_sorted_indices(self):
        """Select top image indices based on rankings for grid creation,
        filtering each tuple by a confidence threshold (sigmoid(confidence) > 0.5).
        For class 0 (real), selects k * (grid_size[0] * grid_size[1] - 1) images,
        and for class 1 (fake), selects k images.
        """
        # Helper function: returns True if sigmoid(confidence) > 0.5.
        def get_conf_mask_v(tup):
            # tup is (img_idx, confidence)
            return torch.tensor(tup[1]).sigmoid().item() > 0.5
    
        k = self.max_grids
        indices = {}
        for cls in [0, 1]:
            # Get the sorted list for this class (list of tuples: (img_idx, confidence))
            cls_list = self.sorted_confs.get(cls, [])
            # Filter the list by confidence threshold.
            filtered = [tup for tup in cls_list if get_conf_mask_v(tup)]
            # Determine the number of required images:
            required = k * (self.grid_size[0] * self.grid_size[1] - 1) if cls == 0 else k
            # Select only the image indices from the filtered list (up to the required number).
            indices[cls] = filtered[:required]
        return indices
    

    def get_image_by_index(self, idx):
        """Return image tensor from dataset by index."""
        if self.dataset is None:
            raise ValueError("Dataset not provided for image retrieval.")
        image, _, _, _ = self.dataset[idx]
        return image

    def save_ranking(self, ranking, file_path):
        with open(file_path, "wb") as f:
            pickle.dump(ranking, f)

    def load_ranking(self, file_path):
        with open(file_path, "rb") as f:
            ranking = pickle.load(f)
        return ranking

    def analysis(self):
        """Evaluate grid tensors and compute overall metrics."""
        results_folder = os.path.join("results", str(self.grid_split), f"{self.model_name}_{self.weights_name}")
        raw_results_file = os.path.join(results_folder, "results.pkl")
        if os.path.exists(raw_results_file):
            raise RuntimeError(f"Results already exist at {raw_results_file}. Use load_results() instead.")

        # List grid tensor files.
        grid_dir = os.path.join(self.output_folder, str(self.grid_split))
        grid_paths = [os.path.join(grid_dir, f) for f in os.listdir(grid_dir) if f.endswith('.pt')]
        logger.info("Found %d grid tensors in %s.", len(grid_paths), grid_dir)

        # Load each grid tensor.
        preprocessed_tensors = [torch.load(path, map_location=self.device) for path in grid_paths]
        logger.info("Loaded all grid tensors.")

        # Choose evaluator based on xai_method.
        if self.xai_method == "bcos":
            evaluator = BCOSEvaluator(self.model, self.device)
        elif self.xai_method == "lime":
            evaluator = LIMEEvaluator(self.model, self.device)
        elif self.xai_method == "gradcam":
            raise NotImplementedError("GradCAM evaluator not implemented.")
        else:
            raise ValueError(f"Unknown xai_method: {self.xai_method}")

        raw_results = evaluator.evaluate(preprocessed_tensors, grid_paths, self.grid_split)
        grid_accuracies = [res["accuracy"] for res in raw_results]
        percentiles = np.percentile(np.array(grid_accuracies), [25, 50, 75, 100])
        logger.info("Localisation accuracy percentiles: %s", percentiles)
        overall = {"localisation_metric": grid_accuracies, "percentiles": percentiles}
        return overall, raw_results

    def create_GPG_grids(self):
        """Create grids by combining ranked real and fake images."""
        logger.info("=== Starting GPG grid creation in %s ===", self.output_folder)
        
        # Check if grids already exist.
        existing_files = [f for f in os.listdir(self.output_dir) if f.endswith('.pt')]
        logger.debug("Found %d existing .pt files in %s.", len(existing_files), self.output_dir)
        if len(existing_files) >= self.max_grids:
            logger.info("Existing grid files found. Skipping grid creation.")
            return
        
        # Copy the ranked real/fake lists
        #ranked_real = self.sorted_confs.get(0, []).copy()
        #ranked_fake = self.sorted_confs.get(1, []).copy()

        indices = self.get_sorted_indices()
        # Expect one fake (class 1) and remaining real (class 0) images.
        ranked_real = indices.get(0, []).copy()
        ranked_fake = indices.get(1, []).copy()
        logger.debug("Ranked real: %d, Ranked fake: %d", len(ranked_real), len(ranked_fake))
        
        n_imgs = self.grid_size[0] * self.grid_size[1]
        logger.debug("Total images per grid: %d", n_imgs)
        side = int(np.sqrt(n_imgs))
        logger.debug("Calculated grid side length: %d", side)
        
        grid_count = 0
        while grid_count < self.max_grids:
            logger.info("--- Creating grid %d of %d ---", grid_count + 1, self.max_grids)
            required_real = n_imgs - 1  # Reserve 1 slot for fake image.
            fake_count = len(ranked_fake)
            real_count = len(ranked_real)
            logger.debug("Need %d real, have %d; need 1 fake, have %d.", required_real, real_count, fake_count)
            
            if fake_count < 1 or real_count < required_real:
                logger.warning("Not enough images: fake %d, real %d (required %d)", fake_count, real_count, required_real)
                break
            
            # Get first fake tuple and remove it.
            fake_tuple = ranked_fake.pop(0)
            logger.info("Selected fake image: idx %d with confidence %.4f", fake_tuple[0], fake_tuple[1])
            fake_img = self.get_image_by_index(fake_tuple[0])
            logger.debug("Fake image shape: %s", fake_img.shape if hasattr(fake_img, 'shape') else "N/A")
            
            # Select first required_real real image tuples.
            selected_real_tuples = ranked_real[:required_real]
            logger.info("Selected real indices: %s", selected_real_tuples)
            ranked_real = ranked_real[required_real:]  # Remove used indices.
            
            # Retrieve real images.
            selected_real = [self.get_image_by_index(idx) for idx, _ in selected_real_tuples]
            logger.debug("Retrieved %d real images.", len(selected_real))
            
            # Combine real and fake images.
            images = selected_real + [fake_img]
            logger.debug("Combined image count: %d", len(images))
            random.shuffle(images)  # Shuffle grid placement.
            logger.debug("Images shuffled.")
            
            # Find fake image index in shuffled list.
            fake_index = next(i for i, img in enumerate(images) if torch.equal(img, fake_img))
            final_fake_index = (fake_index % side) * side + (fake_index // side)
            logger.debug("Fake image: shuffled index %d, final index %d", fake_index, final_fake_index)
            
            # Stack images and reshape into grid tensor.
            stacked = torch.stack(images, dim=0)
            logger.debug("Stacked images shape: %s", stacked.shape)
            grid_tensor = (
                stacked.view(-1, side, side, *stacked.shape[-3:])
                       .permute(0, 3, 2, 4, 1, 5)
                       .reshape(-1, stacked.shape[1], stacked.shape[2] * side, stacked.shape[3] * side)
            )
            logger.debug("Grid tensor shape: %s", grid_tensor.shape)
            
            # Save grid tensor with fake position encoded in filename.
            base_name = f"grid_{grid_count}_fake_{final_fake_index}.pt"
            path_to_grid = os.path.join(self.output_dir, base_name)
            torch.save(grid_tensor, path_to_grid)
            logger.info("Saved grid tensor: %s", path_to_grid)
            
            grid_count += 1
        
        logger.info("=== Finished grid creation. Created %d grids. ===", grid_count)

In [12]:
import argparse

parser = argparse.ArgumentParser(description="Evaluate grids using a model and XAI method.")
parser.add_argument("--base_output_dir", type=str, default="datasets/GPG_grids",
                    help="Base output directory for grids.")
parser.add_argument("--max_grids", type=int, default=3, help="Max number of grids to create.")
parser.add_argument("--xai_method", type=str, default="bcos",
                    choices=["bcos", "lime", "gradcam"], help="XAI method to use.")
parser.add_argument("--model_path", type=str, default=MODEL_PATH,
                    help="Path to model configuration file.")
parser.add_argument("--grid_split", type=int, default=2,
                    help="Grid split (e.g., 3 for a 3x3 grid).")

# Use parse_known_args to ignore the extra kernel arguments that Jupyter passes
args = parser.parse_known_args()[0]
print(args)

Namespace(base_output_dir='datasets/GPG_grids', max_grids=3, xai_method='bcos', model_path='/pfs/data5/home/ma/ma_ma/ma_lineuman/Interpretable-Deep-Fake-Detection/training/config/detector/resnet34_bcos_v2_minimal.yaml', grid_split=2)


In [13]:
grid_size = (args.grid_split, args.grid_split)
logger.info("Parameters: XAI=%s, Base=%s, Model=%s, Grid=%dx%d",
            args.xai_method, args.base_output_dir, args.model_path, args.grid_split, args.grid_split)

INFO:__main__:Parameters: XAI=bcos, Base=datasets/GPG_grids, Model=/pfs/data5/home/ma/ma_ma/ma_lineuman/Interpretable-Deep-Fake-Detection/training/config/detector/resnet34_bcos_v2_minimal.yaml, Grid=2x2


In [14]:
config = load_config(args.model_path, additional_args=ADDITIONAL_ARGS)
model = load_model(config)

pretrained_path = config['pretrained']
state_dict = torch.load(pretrained_path)
# Remove "module." prefix from state_dict keys if necessary.
from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    new_state_dict[k.replace("module.", "")] = v
model.load_state_dict(new_state_dict)

# Set device and move model.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(next(model.parameters()).device)

model.eval()  # Set model to evaluation mode.
logger.info("Loaded model %s on device %s", model.__class__.__name__, device)

model_name = config.get("model_name", "defaultModel")
if not os.path.exists(pretrained_path):
    raise FileNotFoundError(f"Weight file not found: {pretrained_path}")
weights_name = os.path.basename(pretrained_path).split('.')[0]

INFO:__main__:Registered models: ['resnet34', 'resnet34_bcos', 'resnet34_bcos_v2', 'resnet34_bcos_v2_minimal', 'inception_bcos_detector', 'xception', 'vit_bcos', 'convnext_bcos', 'vgg11_v2_bcos']
INFO:training.detectors.resnet34_bcos_v2_minimal_detector:Load pretrained model successfully!
INFO:__main__:Loaded model ResnetBcosDetector_v2_minimal on device cuda


cuda:0


In [15]:
import sys
# Reset sys.argv so that only the script name is present.
sys.argv = [sys.argv[0]]

from train import prepare_testing_data
test_data_loaders = prepare_testing_data(config)
test_loader = list(test_data_loaders.values())[0]

In [16]:
dataset=test_loader.dataset

In [19]:
for i in range(20):
    sample = dataset[i]
    print(sample)
    

(tensor([[[0.2824, 0.2863, 0.2980,  ..., 0.6353, 0.6392, 0.6627],
         [0.2824, 0.2863, 0.2980,  ..., 0.6627, 0.6706, 0.6902],
         [0.2824, 0.2863, 0.2980,  ..., 0.7137, 0.7137, 0.7294],
         ...,
         [0.1333, 0.1333, 0.1333,  ..., 0.1412, 0.1412, 0.1412],
         [0.1333, 0.1333, 0.1333,  ..., 0.1412, 0.1412, 0.1412],
         [0.1333, 0.1333, 0.1333,  ..., 0.1412, 0.1412, 0.1412]],

        [[0.3255, 0.3255, 0.3333,  ..., 0.4314, 0.4431, 0.4549],
         [0.3255, 0.3255, 0.3333,  ..., 0.4745, 0.4784, 0.4863],
         [0.3255, 0.3255, 0.3333,  ..., 0.5216, 0.5255, 0.5333],
         ...,
         [0.1647, 0.1647, 0.1647,  ..., 0.1725, 0.1725, 0.1725],
         [0.1647, 0.1647, 0.1647,  ..., 0.1725, 0.1725, 0.1725],
         [0.1647, 0.1647, 0.1647,  ..., 0.1725, 0.1725, 0.1725]],

        [[0.2941, 0.2941, 0.3020,  ..., 0.1882, 0.1961, 0.2039],
         [0.2941, 0.2941, 0.3020,  ..., 0.2353, 0.2431, 0.2471],
         [0.2941, 0.2941, 0.3020,  ..., 0.2824, 0.2863, 0

In [2]:
# Initialize grid creator with all required objects.
grid_creator = RankedGPGCreator(
    base_output_dir=args.base_output_dir,
    grid_size=grid_size,
    xai_method=args.xai_method,
    max_grids=args.max_grids,
    model=model,
    model_name=model_name,
    weights_name=weights_name,
    test_data_loaders= test_data_loaders,
    #loader=test_loader,
    dataset=test_loader.dataset,
    device=device,
    grid_split=args.grid_split
)

NameError: name 'RankedGPGCreator' is not defined

In [16]:
key = list(test_data_loaders.keys())[0]
for i, data_dict in enumerate(test_data_loaders[key]):
    print(key)
    print(data_dict.keys())
    data, label, mask, landmark, indices = (data_dict[k] for k in ['image', 'label', 'mask', 'landmark', 'index'])
    print(f"Batch of images shape: {img_batch.shape}")


FaceForensics++
dict_keys(['image', 'label', 'landmark', 'mask', 'index'])


NameError: name 'img_batch' is not defined

In [17]:
grid_creator.create_GPG_grids()


INFO:__main__:=== Starting GPG grid creation in datasets/GPG_grids/resnet34_bcos_v2_minimal_bcos_resnet_minimal_b2_ckpt_best ===
INFO:__main__:--- Creating grid 1 of 2 ---
INFO:__main__:=== Finished GPG grid creation. Created 0 grids. ===


In [17]:
grid_creator.run()


INFO:__main__:Found 2 grid tensors in datasets/GPG_grids/resnet34_bcos_v2_minimal_bcos_resnet_minimal_b2_ckpt_best/3ch.
INFO:__main__:Loaded all grid tensors.
INFO:root:Loaded model ResnetBcosDetector_v2_minimal onto cuda
INFO:root:Processing 2 grids with grid_split=3.
INFO:root:Evaluating grid 0 from file: datasets/GPG_grids/resnet34_bcos_v2_minimal_bcos_resnet_minimal_b2_ckpt_best/3ch/grid_1_fake_0.pt
INFO:root:For grid grid_1_fake_0.pt: true position 0, predicted 4, grid accuracy: 0.109
INFO:root:Evaluating grid 1 from file: datasets/GPG_grids/resnet34_bcos_v2_minimal_bcos_resnet_minimal_b2_ckpt_best/3ch/grid_0_fake_7.pt
INFO:root:For grid grid_0_fake_7.pt: true position 7, predicted 1, grid accuracy: 0.108
INFO:__main__:Localisation accuracy percentiles: [0.10847886 0.10866199 0.10884511 0.10902824]
INFO:__main__:Results saved to folder: results/GPG/resnet34_bcos_v2_minimal_bcos_resnet_minimal_b2_ckpt_best


In [ ]:
grid_creator.load_results(load_overall=False)